### Import

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier

### Data Load

In [2]:
train = pd.read_csv('train.csv').drop(columns=['ID'])
test = pd.read_csv('test.csv').drop(columns=['ID'])

missing_cols = set(test.columns) - set(train.columns)
test = test.drop(columns=missing_cols)

In [3]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Columns

In [4]:
# 결측 비율 80% 이상인 컬럼 및 데이터 적은 컬럼 제거
columns_to_drop = [
    "배란 유도 유형", "임신 시도 또는 마지막 임신 경과 연수", "착상 전 유전 검사 사용 여부",
    "PGD 시술 여부", "PGS 시술 여부", "난자 해동 경과일", "배아 해동 경과일"
]

X = X.drop(columns=columns_to_drop, errors='ignore')
test = test.drop(columns=columns_to_drop, errors='ignore')

In [5]:
numeric_columns = [
    "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수", 
    "이식된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수", "미세주입 후 저장된 배아 수",
    "해동된 배아 수", "해동 난자 수", "수집된 신선 난자 수", "저장된 신선 난자 수",
    "혼합된 난자 수", "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일", "난자 혼합 경과일", "배아 이식 경과일"
]

categorical_columns = [
    "시술 시기 코드", "시술 당시 나이", "시술 유형", "특정 시술 유형", "배란 자극 여부",
    "단일 배아 이식 여부", "착상 전 유전 진단 사용 여부", "남성 주 불임 원인",
    "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인", "부부 주 불임 원인",
    "부부 부 불임 원인", "불명확 불임 원인", "불임 원인 - 난관 질환", "불임 원인 - 남성 요인", "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증", "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태", "배아 생성 주요 이유", "총 시술 횟수",
    "클리닉 내 총 시술 횟수", "IVF 시술 횟수", "DI 시술 횟수", "총 임신 횟수", "IVF 임신 횟수", "DI 임신 횟수",
    "총 출산 횟수", "IVF 출산 횟수", "DI 출산 횟수", "난자 출처", "정자 출처", "난자 기증자 나이", "정자 기증자 나이",
    "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부"
]

### Encoding

In [6]:
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)  # 테스트 데이터도 동일하게 변환

In [7]:
# ordinal 인코딩 필요 컬럼
ordinal_columns = [
    "시술 시기 코드", "시술 당시 나이", "총 시술 횟수", "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수", "DI 시술 횟수", "총 임신 횟수", "IVF 임신 횟수", "DI 임신 횟수",
    "총 출산 횟수", "IVF 출산 횟수", "DI 출산 횟수", "난자 기증자 나이", "정자 기증자 나이"
]

# one-hot 인코딩 필요 컬럼
onehot_columns = ["시술 유형"]

# multi label 인코딩 필요 컬럼
multi_label_columns = ["특정 시술 유형", "배아 생성 주요 이유", "난자 출처", "정자 출처"]

In [8]:
X_original = X.copy()
test_original = test.copy()

# 1️. Ordinal Encoding
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_encoded = X.copy()
test_encoded = test.copy()

X_encoded[ordinal_columns] = ordinal_encoder.fit_transform(X[ordinal_columns])
test_encoded[ordinal_columns] = ordinal_encoder.transform(test[ordinal_columns])


# 2️. One-Hot Encoding
onehot_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

onehot_df = pd.DataFrame(onehot_encoder.fit_transform(X_encoded[onehot_columns]),
                         columns=onehot_encoder.get_feature_names_out(onehot_columns))
test_onehot_df = pd.DataFrame(onehot_encoder.transform(test_encoded[onehot_columns]),
                              columns=onehot_encoder.get_feature_names_out(onehot_columns))

# 기존 컬럼 제거 및 변환된 데이터 추가
X_encoded.drop(columns=onehot_columns, inplace=True)
X_encoded = pd.concat([X_encoded, onehot_df], axis=1)
test_encoded.drop(columns=onehot_columns, inplace=True)
test_encoded = pd.concat([test_encoded, test_onehot_df], axis=1)


# 3️. Multi-Label One-Hot Encoding 적용
for col in multi_label_columns:
    X_encoded[col] = X_encoded[col].fillna("Unknown")
    test_encoded[col] = test_encoded[col].fillna("Unknown")  #특정 시술 유형 컬럼 결측값 처리
    
for col in multi_label_columns:
    X_encoded[col] = X_encoded[col].apply(lambda x: eval(x) if isinstance(x, str) and x.startswith("[") else [x])
    test_encoded[col] = test_encoded[col].apply(lambda x: eval(x) if isinstance(x, str) and x.startswith("[") else [x])
    
    mlb = MultiLabelBinarizer()
    X_mlb = mlb.fit_transform(X_encoded[col])
    test_mlb = mlb.transform(test_encoded[col])
    
    multi_df = pd.DataFrame(X_mlb, columns=[f"{col}_{c}" for c in mlb.classes_])
    test_multi_df = pd.DataFrame(test_mlb, columns=[f"{col}_{c}" for c in mlb.classes_])
    
    X_encoded.drop(columns=[col], inplace=True)
    X_encoded = pd.concat([X_encoded, multi_df], axis=1)
    test_encoded.drop(columns=[col], inplace=True)
    test_encoded = pd.concat([test_encoded, test_multi_df], axis=1)

# 최종 인코딩된 데이터셋 출력
print(X_encoded.head())
print(test_encoded.head())

C:\Users\서혜린\AppData\Roaming\Python\Python313\site-packages\sklearn\preprocessing\_label.py:909: UserWarning: unknown class(es) ['IUI:ICI', 'IVF:Unknown:Unknown:Unknown'] will be ignored
  warnings.warn(


   시술 시기 코드  시술 당시 나이 배란 자극 여부 단일 배아 이식 여부 착상 전 유전 진단 사용 여부 남성 주 불임 원인  \
0       6.0       0.0        1         0.0              0.0          0   
1       5.0       5.0        0         0.0              0.0          0   
2       3.0       0.0        1         0.0              0.0          0   
3       2.0       1.0        1         0.0              0.0          0   
4       3.0       0.0        1         0.0              0.0          0   

  남성 부 불임 원인 여성 주 불임 원인 여성 부 불임 원인 부부 주 불임 원인  ... 배아 생성 주요 이유_배아 저장용, 현재 시술용  \
0          0          0          0          0  ...                          0   
1          0          0          0          0  ...                          0   
2          0          0          0          0  ...                          0   
3          0          0          0          0  ...                          0   
4          0          0          0          0  ...                          0   

  배아 생성 주요 이유_연구용, 현재 시술용 배아 생성 주요 이유_현재 시술용 난자 출처_기증 제공 난자 출처_본인 제공

### Pre-Processing

Missing Value

In [9]:
#수치형 변수 결측값 처리

# 중앙값 대체 : 배아 수 관련
median_columns = [
    "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수",
    "이식된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수", "미세주입 후 저장된 배아 수",
    "해동된 배아 수", "수집된 신선 난자 수", "저장된 신선 난자 수",
    "혼합된 난자 수", "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수"
]
X_encoded[median_columns] = X_encoded[median_columns].fillna(X_encoded[median_columns].median())
test_encoded[median_columns] = test_encoded[median_columns].fillna(test_encoded[median_columns].median())

# 평균값 대체 : 시간 관련
mean_columns = ["난자 채취 경과일", "난자 혼합 경과일", "배아 이식 경과일"]
X_encoded[mean_columns] = X_encoded[mean_columns].fillna(X_encoded[mean_columns].mean())
test_encoded[mean_columns] = test_encoded[mean_columns].fillna(test_encoded[mean_columns].mean())

# 0으로 대체 : 해동 난자 수
X_encoded["해동 난자 수"] = X_encoded["해동 난자 수"].fillna(0)
test_encoded["해동 난자 수"] = test_encoded["해동 난자 수"].fillna(0)

Outliers

In [ ]:
from scipy.stats import zscore

# 1️. Winsorization: 극단값을 상위/하위 1% 분위값으로 조정
def winsorize_series(series, lower_quantile=0.01, upper_quantile=0.99):
    lower_bound = series.quantile(lower_quantile)
    upper_bound = series.quantile(upper_quantile)
    return np.clip(series, lower_bound, upper_bound)

for col in numeric_columns:
    if col in X_encoded.columns:
        X_encoded[col] = winsorize_series(X_encoded[col])

# 2️. 로그 변환: 비대칭 데이터 처리
log_transform_cols = ["총 생성 배아 수", "미세주입된 난자 수", "이식된 배아 수", "해동된 배아 수"]
for col in log_transform_cols:
    if col in X_encoded.columns:
        X_encoded[col] = np.log1p(X_encoded[col])  # log(1 + x) 변환 적용

# 3️. Z-score 기반 이상치 처리: 표준편차 3 이상 벗어난 값 조정
zscore_threshold = 3
for col in numeric_columns:
    if col in X_encoded.columns:
        z_scores = zscore(X_encoded[col].dropna())  # 결측값 제외 후 계산
        X_encoded.loc[(z_scores > zscore_threshold) | (z_scores < -zscore_threshold), col] = X_encoded[col].median()

이상치 처리 후 데이터 크기: (256351, 102)


### Sampling

In [11]:

# 첫 실행 시 설치 : pip install imbalanced-learn

from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight

# 클래스 불균형 확인
print("Original class distribution:", Counter(y))

# 데이터 형 변환 (object → numeric)
for col in X_encoded.select_dtypes(include=['object']).columns:
    X_encoded[col] = X_encoded[col].astype('category').cat.codes

# 오버샘플링 : SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_encoded, y)
print("After SMOTE oversampling:", Counter(y_resampled))

# 가중치 조정
class_weights = compute_class_weight("balanced", classes=np.unique(y_resampled), y=y_resampled)
class_weight_dict = {c: w for c, w in zip(np.unique(y_resampled), class_weights)}
print("Computed class weights:", class_weight_dict)


Original class distribution: Counter({0: 190123, 1: 66228})


C:\Users\서혜린\AppData\Roaming\Python\Python313\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] 지정된 파일을 찾을 수 없습니다
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\서혜린\AppData\Roaming\Python\Python313\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "c:\ProgramData\Anaconda3\envs\LGAimers6\Lib\subprocess.py", line 556, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\Anaconda3\envs\LGAimers6\Lib\subprocess.py", line 1038, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
 

After SMOTE oversampling: Counter({0: 190123, 1: 190123})
Computed class weights: {np.int64(0): np.float64(1.0), np.int64(1): np.float64(1.0)}


### Training (Solve Overfitting)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score

# 데이터 분할 (Train:Test = 80:20)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# 모델 학습 (과적합 방지 Hyperparameter 적용)
model = RandomForestClassifier(
    n_estimators=100,         # 트리 개수
    max_depth=10,             # 트리 최대 깊이 제한 (과적합 방지)
    min_samples_split=5,      # 노드를 나누기 위한 최소 샘플 수 (일반적으로 2~10 사이)
    class_weight="balanced",  # 클래스 불균형 보정
    random_state=42
)

# 모델 학습
model.fit(X_train, y_train)

# 예측 수행
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# 확률 예측 (ROC-AUC 계산용)
y_train_proba = model.predict_proba(X_train)[:, 1]
y_test_proba = model.predict_proba(X_test)[:, 1]

# 성능 평가
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
train_roc_auc = roc_auc_score(y_train, y_train_proba)
test_roc_auc = roc_auc_score(y_test, y_test_proba)

print(f"Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")
print(f"Train ROC-AUC: {train_roc_auc:.4f}, Test ROC-AUC: {test_roc_auc:.4f}")

# 교차 검증 수행 (K=5)
cv_scores = cross_val_score(model, X_resampled, y_resampled, cv=5, scoring='roc_auc')
print(f"Cross Validation ROC-AUC Scores: {cv_scores}")
print(f"Mean CV ROC-AUC Score: {cv_scores.mean():.4f}")

Train Accuracy: 0.7357, Test Accuracy: 0.7313
Train ROC-AUC: 0.8256, Test ROC-AUC: 0.8205
Cross Validation ROC-AUC Scores: [0.72983193 0.76637284 0.86229196 0.86165889 0.85708976]
Mean CV ROC-AUC Score: 0.8154


### Predict

In [13]:
pred_proba = model.predict_proba(test_encoded)[:, 1]

### Submission

In [14]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_proba

In [ ]:
sample_submission.to_csv('./enc+smote+preproc_submission.csv', index=False)